In [1]:
import os
import pandas as pd
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)

# Глобальные переменные
TABLE_NAME = "churn_data.csv"  # таблица с данными (или путь к файлу)
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5001
EXPERIMENT_NAME = "customer-churn-prediction"  # название эксперимента
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME = "churn-classifier"  # название зарегистрированной модели

# Настройка подключения к MLflow Tracking Server
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# Установка эксперимента
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment Name: {EXPERIMENT_NAME}")
print(f"Registry Model Name: {REGISTRY_MODEL_NAME}")

MLflow Tracking URI: http://127.0.0.1:5001
Experiment Name: customer-churn-prediction
Registry Model Name: churn-classifier


In [2]:
import psycopg
import pandas as pd
from dotenv import load_dotenv

# Параметры подключения к PostgreSQL (замените на свои)
load_dotenv('../.env')

TABLE_NAME = "users_churn"

# Загрузка данных
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}
connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

print(f"✅ Загружено {len(df)} строк из таблицы {TABLE_NAME}")
print(f"Размерность данных: {df.shape}")
print("\nПервые строки:")
print(df.head())

✅ Загружено 7043 строк из таблицы users_churn
Размерность данных: (7043, 22)

Первые строки:
     id customer_id begin_date   end_date            type paperless_billing  \
0  2744  9637-CDTKZ 2014-02-01        NaT        Two year               Yes   
1  2746  3946-JEWRQ 2016-03-01        NaT        One year               Yes   
2  2748  7873-CVMAW 2014-02-01        NaT        Two year                No   
3  2750  0463-WZZKO 2019-11-01        NaT  Month-to-month               Yes   
4  2752  3494-JCHRQ 2019-12-01 2020-01-01  Month-to-month               Yes   

            payment_method  monthly_charges  total_charges internet_service  \
0  Credit card (automatic)           114.10        8086.40      Fiber optic   
1  Credit card (automatic)            95.20        4563.00      Fiber optic   
2  Credit card (automatic)            88.55        6362.35              DSL   
3  Credit card (automatic)            20.75          67.10             None   
4         Electronic check           

In [3]:
# Сначала создаём obj_df - датафрейм с категориальными колонками
obj_df = df.select_dtypes(include="object")

print(f"Категориальные колонки: {obj_df.columns.tolist()}")
print(f"Форма obj_df: {obj_df.shape}\n")

# определение категориальных колонок, которые будут преобразованы
cat_columns = ["type", "payment_method", "internet_service", "gender"]

# создание объекта OneHotEncoder для преобразования категориальных переменных
# auto - автоматическое определение категорий
# ignore - игнорировать ошибки, если встречается неизвестная категория
# max_categories - максимальное количество уникальных категорий
# sparse_output - вывод в виде разреженной матрицы, если False, то в виде обычного массива
# drop="first" - удаляет первую категорию, чтобы избежать ловушки мультиколлинеарности
encoder_oh = OneHotEncoder(
    categories='auto',
    handle_unknown='ignore',
    max_categories=10,
    sparse_output=False,
    drop='first'
)

# применение OneHotEncoder к данным. Преобразование категориальных данных в массив
encoded_features = encoder_oh.fit_transform(df[cat_columns])

# преобразование полученных признаков в DataFrame и установка названий колонок
# get_feature_names_out() - получение имён признаков после преобразования
encoded_df = pd.DataFrame(
    encoded_features,
    columns=encoder_oh.get_feature_names_out(cat_columns),
    index=obj_df.index
)

# конкатенация исходного DataFrame с новым DataFrame, содержащим закодированные категориальные признаки
# axis=1 означает конкатенацию по колонкам
obj_df = pd.concat([obj_df, encoded_df], axis=1)

print("Результат:")
obj_df.head(2)

Категориальные колонки: ['customer_id', 'type', 'paperless_billing', 'payment_method', 'internet_service', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'gender', 'partner', 'dependents', 'multiple_lines']
Форма obj_df: (7043, 15)

Результат:


,customer_id,type,paperless_billing,payment_method,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,...,dependents,multiple_lines,type_One year,type_Two year,payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,internet_service_Fiber optic,internet_service_None,gender_Male
0,9637-CDTKZ,Two year,Yes,Credit card (automatic),Fiber optic,Yes,Yes,Yes,Yes,Yes,...,No,Yes,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
1,3946-JEWRQ,One year,Yes,Credit card (automatic),Fiber optic,No,No,No,No,Yes,...,No,Yes,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [4]:
from sklearn.impute import SimpleImputer

num_columns = ["monthly_charges", "total_charges"]
n_knots = 3
degree_spline = 4
n_quantiles = 100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

# Создаём num_df - датафрейм с числовыми колонками
num_df = df.select_dtypes(include=['int64', 'float64'])

print(f"Числовые колонки: {num_df.columns.tolist()}")
print(f"Форма num_df: {num_df.shape}")

# Проверим наличие пропусков
print("\nПропуски в числовых колонках:")
print(df[num_columns].isnull().sum())
print()

# Обработка пропусков - заполняем медианой
imputer = SimpleImputer(strategy='median')
df[num_columns] = imputer.fit_transform(df[num_columns])

print("После обработки пропусков:")
print(df[num_columns].isnull().sum())
print()

# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy())
encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_spl.get_feature_names_out(num_columns)
)
num_df = pd.concat([num_df, encoded_df], axis=1)

# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles, subsample=subsample)
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy())
encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# RobustScaler
encoder_rb = RobustScaler()
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy())
encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree)
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy())
encoded_df = pd.DataFrame(encoded_features, columns=encoder_pol.get_feature_names_out(num_columns))
# get all columns after the intercept and original features
encoded_df = encoded_df.iloc[:, 1 + len(num_columns):]
encoded_df.columns = encoder_pol.get_feature_names_out(num_columns)[1 + len(num_columns):]
num_df = pd.concat([num_df, encoded_df], axis=1)

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy)
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy())
encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

print("\nРезультат:")
num_df.head(2)

Числовые колонки: ['id', 'monthly_charges', 'total_charges', 'senior_citizen', 'target']
Форма num_df: (7043, 5)

Пропуски в числовых колонках:
monthly_charges     0
total_charges      11
dtype: int64

После обработки пропусков:
monthly_charges    0
total_charges      0
dtype: int64


Результат:


,id,monthly_charges,total_charges,senior_citizen,target,monthly_charges_sp_0,monthly_charges_sp_1,monthly_charges_sp_2,monthly_charges_sp_3,monthly_charges_sp_4,...,total_charges_robust,monthly_charges^2,monthly_charges total_charges,total_charges^2,monthly_charges^3,monthly_charges^2 total_charges,monthly_charges total_charges^2,total_charges^3,monthly_charges_bin,total_charges_bin
0,2744,114.1,8086.4,0,0,0.0,0.000003,0.059350,0.502083,0.410308,...,1.976414,13018.81,922658.24,65389864.96,1485446.221,1.052753e+08,7.460984e+09,5.287686e+11,4.0,4.0
1,2746,95.2,4563.0,0,0,0.0,0.002010,0.183802,0.598345,0.212523,...,0.935335,9063.04,434397.60,20820969.00,862801.408,4.135465e+07,1.982156e+09,9.500608e+10,3.0,2.0


Применять по отдельности каждое преобразование не всегда удобно, особенно когда их очень много. Поэтому их объединяют в пайплайн или колоночные преобразования, которые можно визуализировать, запустить одной командой и сохранить. Последуйте этому примеру и объедините все энкодеры в одно преобразование. 

### Задание 3
Напишите код, который объединит преобразования над числовыми колонками в ColumnTransformer, а над категориальными — в Pipeline, используя энкодеры из предыдущих заданий. Затем объедините два получившихся объекта класса одним колоночным преобразованием. После чего объедините ваш преобразованный набор данных с изначальным, а результат сохраните в переменную df. 

In [5]:
from sklearn.impute import SimpleImputer

# Обработка пропусков перед преобразованиями
imputer = SimpleImputer(strategy='median')
df[num_columns] = imputer.fit_transform(df[num_columns])

numeric_transformer = ColumnTransformer(transformers=[('spl', encoder_spl, num_columns), ('q', encoder_q, num_columns), ('rb', encoder_rb, num_columns), ('pol', encoder_pol, num_columns), ('kbd', encoder_kbd, num_columns)])

categorical_transformer = Pipeline(steps=[('encoder', encoder_oh)])

preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, num_columns), ('cat', categorical_transformer, cat_columns)], n_jobs=-1)
encoded_features = preprocessor.fit_transform(df)

# Получаем имена признаков после преобразований
feature_names = preprocessor.get_feature_names_out()

transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out())

df = pd.concat([df, transformed_df], axis=1)

df.head(2)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,num__kbd__monthly_charges,num__kbd__total_charges,cat__type_One year,cat__type_Two year,cat__payment_method_Credit card (automatic),cat__payment_method_Electronic check,cat__payment_method_Mailed check,cat__internet_service_Fiber optic,cat__internet_service_None,cat__gender_Male
0,2744,9637-CDTKZ,2014-02-01,NaT,Two year,Yes,Credit card (automatic),114.1,8086.4,Fiber optic,...,4.0,4.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
1,2746,3946-JEWRQ,2016-03-01,NaT,One year,Yes,Credit card (automatic),95.2,4563.0,Fiber optic,...,3.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


### Задание 4
Чтобы визуализировать получившееся общее преобразование, просто посмотрите на значение переменной preprocessor.
Объект preprocessor класса ColumnTransformer, объявленный в предыдущем задании, можно сохранить в MLflow, в директорию preprocessor — точно так же, как и модель.

In [6]:
import os
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from dotenv import load_dotenv
import shutil
import tempfile
import pickle

# Загружаем .env файл
load_dotenv('../.env')

# КРИТИЧЕСКИ ВАЖНО: Настройка S3 ДО любых операций с MLflow
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("S3_ACCESS_KEY")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("S3_SECRET_KEY")

# Проверка переменных окружения
print("Проверка переменных окружения:")
print(f"MLFLOW_S3_ENDPOINT_URL: {os.environ.get('MLFLOW_S3_ENDPOINT_URL')}")
print(f"AWS_ACCESS_KEY_ID: {'Установлена' if os.environ.get('AWS_ACCESS_KEY_ID') else 'НЕ УСТАНОВЛЕНА ❌'}")
print(f"AWS_SECRET_ACCESS_KEY: {'Установлена' if os.environ.get('AWS_SECRET_ACCESS_KEY') else 'НЕ УСТАНОВЛЕНА ❌'}")

# Проверяем, что все credentials установлены
if not os.environ.get('AWS_ACCESS_KEY_ID') or not os.environ.get('AWS_SECRET_ACCESS_KEY'):
    raise ValueError("❌ AWS credentials НЕ установлены! Проверьте файл .env")

print("✅ Все переменные окружения установлены корректно\n")

# ВАЖНО: Пересоздаём MLflow клиент с новыми переменными окружения
import importlib
importlib.reload(mlflow)
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_experiment(EXPERIMENT_NAME)

# Получение experiment_id
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
experiment_id = experiment.experiment_id
print(f"✅ Найден эксперимент '{EXPERIMENT_NAME}' с ID: {experiment_id}\n")

# Создаём пример входных данных для сигнатуры
input_example = df[num_columns + cat_columns].head(5)

# Преобразуем данные для получения выходного примера
output_example = preprocessor.transform(input_example)

# Создаём сигнатуру модели
signature = infer_signature(input_example, output_example)

print("Подготовка данных для сохранения:")
print(f"  Input shape: {input_example.shape}")
print(f"  Output shape: {output_example.shape}")
print(f"  Signature: {signature}\n")

# Запуск с правильным сохранением через log_artifact
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    current_run_id = run.info.run_id
    
    print(f"✅ Run ID: {current_run_id}")
    print(f"✅ Run Name: {RUN_NAME}")
    print(f"✅ Artifact URI: {run.info.artifact_uri}\n")
    
    # Логирование параметров
    mlflow.log_param("num_columns", str(num_columns))
    mlflow.log_param("cat_columns", str(cat_columns))
    mlflow.log_param("n_knots", n_knots)
    mlflow.log_param("degree_spline", degree_spline)
    mlflow.log_param("n_quantiles", n_quantiles)
    mlflow.log_param("polynomial_degree", degree)
    mlflow.log_param("n_bins", n_bins)
    mlflow.log_param("total_features", output_example.shape[1])
    
    # Сохранение preprocessor через директорию
    print("Сохранение preprocessor в MLflow через log_artifact...")
    try:
        # Создаём временную директорию для сохранения модели
        with tempfile.TemporaryDirectory() as tmpdir:
            model_path = os.path.join(tmpdir, "column_transformer")
            
            # Используем mlflow.sklearn.save_model для сохранения локально
            mlflow.sklearn.save_model(
                sk_model=preprocessor,
                path=model_path,
                signature=signature,
                input_example=input_example
            )
            
            # Логируем всю директорию как артефакт
            mlflow.log_artifacts(model_path, artifact_path="column_transformer")
            
        print(f"\n✅ Preprocessor успешно сохранён через log_artifacts!")
        
    except Exception as e:
        print(f"\n❌ ОШИБКА при сохранении: {e}")
        import traceback
        traceback.print_exc()
        raise
    
    print(f"Artifact URI: {mlflow.get_artifact_uri()}")
    print(f"View run: http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}/#/experiments/{experiment_id}/runs/{current_run_id}")

# Сохраняем run_id для дальнейшего использования
run_id = current_run_id

# Проверка загрузки из S3
print("\n" + "="*70)
print("Проверка загрузки preprocessor...\n")

# Даём время на загрузку в S3
import time
print("Ожидание 3 секунды для завершения загрузки в S3...")
time.sleep(3)

try:
    # Загружаем из MLflow
    model_uri = f"runs:/{run_id}/column_transformer"
    print(f"Загрузка из: {model_uri}")
    
    loaded_preprocessor = mlflow.sklearn.load_model(model_uri)
    
    # Тестируем загруженный preprocessor
    test_data = df[num_columns + cat_columns].head(3)
    test_result = loaded_preprocessor.transform(test_data)
    
    print(f"\n✅ Preprocessor успешно загружен и работает!")
    print(f"Тестовая трансформация: {test_data.shape} -> {test_result.shape}")
    
except Exception as e:
    print(f"\n❌ Ошибка при загрузке: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*70)
print(f"\n📝 Ваш run_id:")
print(f'run_id = "{run_id}"')

# Финальная проверка через boto3
print("\n" + "="*70)
print("Финальная проверка S3 через boto3...\n")

try:
    import boto3
    
    s3_client = boto3.client(
        's3',
        endpoint_url=os.environ.get('MLFLOW_S3_ENDPOINT_URL'),
        aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY'),
        region_name='ru-central1'
    )
    
    bucket_name = os.getenv('S3_BUCKET_NAME')
    prefix = f"5/{run_id}/artifacts/"
    
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        print(f"✅ В S3 найдено {len(response['Contents'])} файлов:")
        for obj in response['Contents']:
            print(f"  - {obj['Key']} ({obj['Size']} bytes)")
    else:
        print(f"❌ В S3 НЕТ файлов по пути {prefix}")
        print("\n⚠️  ВАЖНО: Если файлов нет в S3, но загрузка работает,")
        print("  значит MLflow использует локальное хранилище.")
        print("  Артефакты сохраняются, но не в S3.")
        
except Exception as e:
    print(f"⚠️  Не удалось проверить S3: {e}")
    import traceback
    traceback.print_exc()

Проверка переменных окружения:
MLFLOW_S3_ENDPOINT_URL: https://storage.yandexcloud.net
AWS_ACCESS_KEY_ID: Установлена
AWS_SECRET_ACCESS_KEY: Установлена
✅ Все переменные окружения установлены корректно

✅ Найден эксперимент 'customer-churn-prediction' с ID: 5

Подготовка данных для сохранения:
  Input shape: (5, 6)
  Output shape: (5, 36)
  Signature: inputs: 
  ['monthly_charges': double (required), 'total_charges': double (required), 'type': string (required), 'payment_method': string (required), 'internet_service': string (optional), 'gender': string (required)]
outputs: 
  [Tensor('float64', (-1, 36))]
params: 
  None


✅ Run ID: 303d5cf950a544f9980fe5311ad3b804
✅ Run Name: preprocessing
✅ Artifact URI: s3://s3-student-mle-20250507-39f5f3ff21-freetrack/5/303d5cf950a544f9980fe5311ad3b804/artifacts



2025/12/05 14:25:33 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


Сохранение preprocessor в MLflow через log_artifact...

✅ Preprocessor успешно сохранён через log_artifacts!
Artifact URI: s3://s3-student-mle-20250507-39f5f3ff21-freetrack/5/303d5cf950a544f9980fe5311ad3b804/artifacts
View run: http://127.0.0.1:5001/#/experiments/5/runs/303d5cf950a544f9980fe5311ad3b804
🏃 View run preprocessing at: http://127.0.0.1:5001/#/experiments/5/runs/303d5cf950a544f9980fe5311ad3b804
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/5

Проверка загрузки preprocessor...

Ожидание 3 секунды для завершения загрузки в S3...
Загрузка из: runs:/303d5cf950a544f9980fe5311ad3b804/column_transformer



✅ Preprocessor успешно загружен и работает!
Тестовая трансформация: (3, 6) -> (3, 36)


📝 Ваш run_id:
run_id = "303d5cf950a544f9980fe5311ad3b804"

Финальная проверка S3 через boto3...

✅ В S3 найдено 7 файлов:
  - 5/303d5cf950a544f9980fe5311ad3b804/artifacts/column_transformer/MLmodel (1031 bytes)
  - 5/303d5cf950a544f9980fe5311ad3b804/artifacts/column_transformer/conda.yaml (209 bytes)
  - 5/303d5cf950a544f9980fe5311ad3b804/artifacts/column_transformer/input_example.json (495 bytes)
  - 5/303d5cf950a544f9980fe5311ad3b804/artifacts/column_transformer/model.pkl (13080 bytes)
  - 5/303d5cf950a544f9980fe5311ad3b804/artifacts/column_transformer/python_env.yaml (120 bytes)
  - 5/303d5cf950a544f9980fe5311ad3b804/artifacts/column_transformer/requirements.txt (94 bytes)
  - 5/303d5cf950a544f9980fe5311ad3b804/artifacts/column_transformer/serving_input_example.json (890 bytes)


### Задание 5
Теперь обучите новую модель с новыми признаками, полученными при помощи ваших энкодеров. Посмотрите, как изменилось качество модели на валидационной или тестовой выборке. Зарегистрируйте полученную версию модели.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# Подготовка данных
print("="*70)
print("ОБУЧЕНИЕ МОДЕЛИ С НОВЫМИ ПРИЗНАКАМИ")
print("="*70)

# Выделяем целевую переменную
y = df['target'].values

# Преобразуем признаки с помощью preprocessor
X_transformed = preprocessor.transform(df[num_columns + cat_columns])

print(f"\n📊 Размерность данных:")
print(f"  Исходные признаки: {len(num_columns + cat_columns)}")
print(f"  После преобразования: {X_transformed.shape[1]}")
print(f"  Количество примеров: {X_transformed.shape[0]}")
print(f"  Распределение классов: {np.bincount(y)}")

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_transformed, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"\n📦 Разделение данных:")
print(f"  Train: {X_train.shape[0]} примеров")
print(f"  Test: {X_test.shape[0]} примеров")

# Обучение модели
print(f"\n🤖 Обучение RandomForestClassifier...")

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

print(f"✅ Модель обучена!")

# Предсказания
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
y_test_proba = model.predict_proba(X_test)[:, 1]

# Метрики
train_metrics = {
    "accuracy": accuracy_score(y_train, y_train_pred),
    "precision": precision_score(y_train, y_train_pred),
    "recall": recall_score(y_train, y_train_pred),
    "f1": f1_score(y_train, y_train_pred)
}

test_metrics = {
    "accuracy": accuracy_score(y_test, y_test_pred),
    "precision": precision_score(y_test, y_test_pred),
    "recall": recall_score(y_test, y_test_pred),
    "f1": f1_score(y_test, y_test_pred),
    "roc_auc": roc_auc_score(y_test, y_test_proba)
}

print(f"\n📈 Метрики на обучающей выборке:")
for metric_name, metric_value in train_metrics.items():
    print(f"  {metric_name}: {metric_value:.4f}")

print(f"\n📈 Метрики на тестовой выборке:")
for metric_name, metric_value in test_metrics.items():
    print(f"  {metric_name}: {metric_value:.4f}")

print(f"\n✅ Модель готова к регистрации!")

ОБУЧЕНИЕ МОДЕЛИ С НОВЫМИ ПРИЗНАКАМИ

📊 Размерность данных:
  Исходные признаки: 6
  После преобразования: 36
  Количество примеров: 7043
  Распределение классов: [5174 1869]

📦 Разделение данных:
  Train: 5634 примеров
  Test: 1409 примеров

🤖 Обучение RandomForestClassifier...
✅ Модель обучена!

📈 Метрики на обучающей выборке:
  accuracy: 0.8683
  precision: 0.8365
  recall: 0.6261
  f1: 0.7161

📈 Метрики на тестовой выборке:
  accuracy: 0.7991
  precision: 0.6730
  recall: 0.4733
  f1: 0.5557
  roc_auc: 0.8356

✅ Модель готова к регистрации!


In [8]:
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
import tempfile
import os
import boto3
import traceback

# Убедимся, что переменные окружения установлены
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("S3_ACCESS_KEY")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("S3_SECRET_KEY")

# Настройка MLflow
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_experiment(EXPERIMENT_NAME)

print("="*70)
print("ЛОГИРОВАНИЕ И РЕГИСТРАЦИЯ МОДЕЛИ В MLFLOW")
print("="*70)

# Создаём input example для сигнатуры
input_example = X_test[:5]
signature = infer_signature(input_example, model.predict(input_example))

# Запускаем новый run для модели
with mlflow.start_run(run_name="model_training_with_features") as run:
    model_run_id = run.info.run_id
    
    print(f"\n✅ Run ID: {model_run_id}")
    print(f"✅ Artifact URI: {run.info.artifact_uri}\n")
    
    # Логируем параметры модели
    mlflow.log_param("model_type", "RandomForestClassifier")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 10)
    mlflow.log_param("min_samples_split", 5)
    mlflow.log_param("min_samples_leaf", 2)
    mlflow.log_param("n_features", X_transformed.shape[1])
    
    # Логируем метрики
    print("📊 Логирование метрик...")
    for metric_name, metric_value in train_metrics.items():
        mlflow.log_metric(f"train_{metric_name}", metric_value)
    
    for metric_name, metric_value in test_metrics.items():
        mlflow.log_metric(f"test_{metric_name}", metric_value)
    
    print("✅ Метрики залогированы")
    
    # Сохраняем модель через log_artifacts (как с preprocessor)
    print("\n💾 Сохранение модели в MLflow...")
    
    try:
        with tempfile.TemporaryDirectory() as tmpdir:
            model_path = os.path.join(tmpdir, REGISTRY_MODEL_NAME)
            
            # Сохраняем модель локально
            mlflow.sklearn.save_model(
                sk_model=model,
                path=model_path,
                signature=signature,
                input_example=input_example
            )
            
            # Логируем как артефакт
            mlflow.log_artifacts(model_path, artifact_path=REGISTRY_MODEL_NAME)
            
        print(f"✅ Модель успешно сохранена!")
        
    except Exception as e:
        print(f"❌ ОШИБКА при сохранении модели: {e}")
        traceback.print_exc()
        raise

print(f"\n🔗 View run: http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}/#/experiments/{experiment.experiment_id}/runs/{model_run_id}")

# Регистрация модели в Model Registry
print(f"\n{'='*70}")
print("РЕГИСТРАЦИЯ МОДЕЛИ В MODEL REGISTRY")
print("="*70)

model_uri = f"runs:/{model_run_id}/{REGISTRY_MODEL_NAME}"

print(f"\n📝 Регистрация модели...")
print(f"  Model URI: {model_uri}")
print(f"  Model Name: {REGISTRY_MODEL_NAME}")

try:
    # Регистрируем модель
    model_version = mlflow.register_model(
        model_uri=model_uri,
        name=REGISTRY_MODEL_NAME,
        tags={"features": "transformed", "preprocessor": "column_transformer"}
    )
    
    print(f"\n✅ Модель зарегистрирована!")
    print(f"  Model Name: {model_version.name}")
    print(f"  Version: {model_version.version}")
    print(f"  Run ID: {model_version.run_id}")
    
    # Сохраняем информацию для задания
    model_version_id = model_version.version
    model_registred_name = model_version.name
    model_run_id_final = model_version.run_id
    
except Exception as e:
    print(f"\n❌ ОШИБКА при регистрации модели: {e}")
    traceback.print_exc()

# Проверка S3
print(f"\n{'='*70}")
print("ПРОВЕРКА СОХРАНЕНИЯ В S3")
print("="*70)

import time
print("\nОжидание 3 секунды...")
time.sleep(3)

try:
    s3_client = boto3.client(
        's3',
        endpoint_url=os.environ.get('MLFLOW_S3_ENDPOINT_URL'),
        aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY'),
        region_name='ru-central1'
    )
    
    bucket_name = os.getenv('S3_BUCKET_NAME')
    prefix = f"5/{model_run_id}/artifacts/"
    
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        print(f"\n✅ В S3 найдено {len(response['Contents'])} файлов:")
        for obj in response['Contents'][:10]:  # Показываем первые 10
            print(f"  - {obj['Key']} ({obj['Size']} bytes)")
        if len(response['Contents']) > 10:
            print(f"  ... и ещё {len(response['Contents']) - 10} файлов")
    else:
        print(f"\n⚠️  В S3 пока нет файлов (они могут быть в процессе загрузки)")
        
except Exception as e:
    print(f"\n⚠️  Ошибка проверки S3: {e}")

print(f"\n{'='*70}")
print("ИТОГОВАЯ ИНФОРМАЦИЯ ДЛЯ ЗАДАНИЯ 5")
print("="*70)
print(f"\nmodel_version_id = {model_version_id}")
print(f'model_registred_name = "{model_registred_name}"')
print(f'run_id = "{model_run_id_final}"')

ЛОГИРОВАНИЕ И РЕГИСТРАЦИЯ МОДЕЛИ В MLFLOW

✅ Run ID: 83f3cab20d5f4055b65f801167e942f2
✅ Artifact URI: s3://s3-student-mle-20250507-39f5f3ff21-freetrack/5/83f3cab20d5f4055b65f801167e942f2/artifacts

📊 Логирование метрик...
✅ Метрики залогированы

💾 Сохранение модели в MLflow...
✅ Модель успешно сохранена!
🏃 View run model_training_with_features at: http://127.0.0.1:5001/#/experiments/5/runs/83f3cab20d5f4055b65f801167e942f2
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/5

🔗 View run: http://127.0.0.1:5001/#/experiments/5/runs/83f3cab20d5f4055b65f801167e942f2

РЕГИСТРАЦИЯ МОДЕЛИ В MODEL REGISTRY

📝 Регистрация модели...
  Model URI: runs:/83f3cab20d5f4055b65f801167e942f2/churn-classifier
  Model Name: churn-classifier


Successfully registered model 'churn-classifier'.
2025/12/05 14:30:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: churn-classifier, version 1
Created version '1' of model 'churn-classifier'.



✅ Модель зарегистрирована!
  Model Name: churn-classifier
  Version: 1
  Run ID: 83f3cab20d5f4055b65f801167e942f2

ПРОВЕРКА СОХРАНЕНИЯ В S3

Ожидание 3 секунды...

✅ В S3 найдено 7 файлов:
  - 5/83f3cab20d5f4055b65f801167e942f2/artifacts/churn-classifier/MLmodel (886 bytes)
  - 5/83f3cab20d5f4055b65f801167e942f2/artifacts/churn-classifier/conda.yaml (209 bytes)
  - 5/83f3cab20d5f4055b65f801167e942f2/artifacts/churn-classifier/input_example.json (2477 bytes)
  - 5/83f3cab20d5f4055b65f801167e942f2/artifacts/churn-classifier/model.pkl (4201880 bytes)
  - 5/83f3cab20d5f4055b65f801167e942f2/artifacts/churn-classifier/python_env.yaml (120 bytes)
  - 5/83f3cab20d5f4055b65f801167e942f2/artifacts/churn-classifier/requirements.txt (94 bytes)
  - 5/83f3cab20d5f4055b65f801167e942f2/artifacts/churn-classifier/serving_input_example.json (3627 bytes)

ИТОГОВАЯ ИНФОРМАЦИЯ ДЛЯ ЗАДАНИЯ 5

model_version_id = 1
model_registred_name = "churn-classifier"
run_id = "83f3cab20d5f4055b65f801167e942f2"


In [9]:
# Задание 5: Итоговые значения для проверки

model_version_id = model_version_id  # номер зарегистрированной модели
model_registred_name = model_registred_name  # название зарегистрированной модели
run_id = model_run_id_final  # run_id, в рамках которого была зарегистрирована модель

print("✅ Значения для проверки задания:")
print(f"model_version_id = {model_version_id}")
print(f'model_registred_name = "{model_registred_name}"')
print(f'run_id = "{run_id}"')

✅ Значения для проверки задания:
model_version_id = 1
model_registred_name = "churn-classifier"
run_id = "83f3cab20d5f4055b65f801167e942f2"
